# Neural Machine Translation with RNN

We apply RNN to the language translation task, which is a typical seq to seq learning tastk. We will use a Encoder-Decoder architecture, where we use RNN models for both Encoder and Decoder parts. 

We will use the data from [the Tatoeba Project](http://www.manythings.org/anki/) and specifically, we will start with the fra-eng (French-English) translation data.

# Data Processing

In [249]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from random import shuffle, sample
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

## Read raw texts and perform train-val-test splitting

In [250]:
## Read all lines and shuffle
file = open("datasets/fra-eng/fra.txt", "r")
all_lines = file.readlines()
shuffle(all_lines)
#all_lines = sample(all_lines,100000)
print(len(all_lines))
## Train-Val-Test split
train_ratio, val_ratio = 0.8, 0.1
train_end = int(len(all_lines)*train_ratio)
val_end = int(len(all_lines)*(train_ratio+val_ratio))
train_raw = all_lines[:train_end]
val_raw = all_lines[train_end:val_end]
test_raw = all_lines[val_end:]

197463


## Vocab, Dataset and Dataloader

In [251]:
def get_vocabs(lines, source_tokenizer, target_tokenizer):
    source_counter = Counter()
    target_counter = Counter()
    for line in lines:
        parts = line.split("\t") #source, target and note are separated by tab
        source_counter.update(source_tokenizer(parts[0].lower()))
        target_counter.update(target_tokenizer(parts[1].lower()))
    source_vocab = vocab(source_counter, specials=["<unk>","<pad>", "<bos>", "<eos>"])
    target_vocab = vocab(target_counter, specials=["<unk>","<pad>", "<bos>", "<eos>"])
    source_vocab.set_default_index(source_vocab['<unk>'])
    target_vocab.set_default_index(target_vocab['<unk>'])    
    return source_vocab, target_vocab

def process_data(lines, source_tokenizer, target_tokenizer, source_vocab, target_vocab):
    data=[]
    for line in lines:
        source, target, _ = line.split("\t")
        source_tensor = torch.tensor(source_vocab(source_tokenizer(source.lower())))
        target_tensor = torch.tensor(target_vocab(target_tokenizer(target.lower())))
        data.append((source_tensor, target_tensor))
    return data

def generate_batch(data_batch, source_vocab, target_vocab):
    source_batch = []
    target_batch = []
    for source, target in data_batch:
        source_batch.append(torch.cat([torch.tensor([source_vocab["<bos>"]]), source, torch.tensor([source_vocab["<eos>"]])], dim=0))
        target_batch.append(torch.cat([torch.tensor([target_vocab["<bos>"]]), target, torch.tensor([target_vocab["<eos>"]])], dim=0))
    source_batch = pad_sequence(source_batch, padding_value=source_vocab["<pad>"]).pin_memory()
    target_batch = pad_sequence(target_batch, padding_value=target_vocab["<pad>"]).pin_memory()
    return source_batch, target_batch

In [252]:
## Use tokenizers from spacy
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
fr_tokenizer = get_tokenizer("spacy", language="fr_core_news_sm")
## Get source target vocabs
en_vocab, fr_vocab = get_vocabs(train_raw, en_tokenizer, fr_tokenizer)
## Get source tartget token ids
train_data = process_data(train_raw, en_tokenizer, fr_tokenizer, en_vocab, fr_vocab)
val_data = process_data(val_raw, en_tokenizer, fr_tokenizer, en_vocab, fr_vocab)
test_data = process_data(test_raw, en_tokenizer, fr_tokenizer, en_vocab, fr_vocab)
## Get train, val, test dataloader
collator = lambda x: generate_batch(x, en_vocab, fr_vocab)
bsz = 256
train_iter = DataLoader(train_data, batch_size=bsz, shuffle=True, collate_fn=collator)
val_iter = DataLoader(val_data, batch_size=bsz, shuffle=False, collate_fn=collator)
test_iter = DataLoader(test_data, batch_size=bsz, shuffle=False, collate_fn=collator)

# Building Seq2Seq Module

In [253]:
from torch import nn 
import torch.nn.functional as F 
import random

In [254]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(2*enc_hid_dim, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embeded = self.embedding(src)
        outputs, hidden = self.rnn(embeded)
        hidden = torch.tanh(self.fc(torch.cat([hidden[-2,:,:],hidden[-1,:,:]],dim=-1)))

        return outputs, hidden

class Attention(nn.Module):
    def __init__(self,enc_hid_dim, dec_hid_dim, attn_dim):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim*2) + dec_hid_dim 
        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self, decoder_hidden, encoder_outputs):
        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1,0,2)

        energy = torch.tanh(
            self.attn(torch.cat([repeated_decoder_hidden, encoder_outputs], dim=-1))
        )
        attention = torch.sum(energy, dim=-1)
        return F.softmax(attention, dim=-1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(enc_hid_dim*2+emb_dim, dec_hid_dim)
        self.out = nn.Linear(self.attention.attn_in+emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def _weighted_encoder_rep(self, decoder_hidden, encoder_outputs):
        a = self.attention(decoder_hidden, encoder_outputs)
        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1,0,2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1,0,2)

        return weighted_encoder_rep
    
    def forward(self, input, decoder_hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embeded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)

        rnn_input = torch.cat((embeded, weighted_encoder_rep),dim=2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embeded = embeded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output, weighted_encoder_rep, embeded),dim=1))
        return output, decoder_hidden.squeeze(0)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1] # tensor.max() returns tuple of maxiums and idmax
            output = (trg[t] if teacher_force else top1)
        return outputs

In [255]:
input_dim = len(en_vocab)
emb_dim = 32
enc_hid_dim = 64
dec_hid_dim = 64
dropout = 0.8
encoder = Encoder(input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout)
attn_dim = 8
attention = Attention(enc_hid_dim, dec_hid_dim, attn_dim)
output_dim = len(fr_vocab)
decoder = Decoder(output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention)

##device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")  ## Only works on M-Series Mac
device = torch.device("cuda")## Only works on NVIDIA devices
model = Seq2Seq(encoder, decoder,device).to(device)

In [256]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(14527, 32)
    (rnn): GRU(32, 64, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=192, out_features=8, bias=True)
    )
    (embedding): Embedding(24739, 32)
    (rnn): GRU(160, 64)
    (out): Linear(in_features=224, out_features=24739, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
  )
)

# Train, Evaluation, Prediction

## Training and Evaluation

In [257]:
from torch import optim
import math

In [258]:
def train_step(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        output = model(src, trg)

        output = output[1:,].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        optimizer.zero_grad()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

def evaluate_step(model, iterator, criterion):
    model.eval
    
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        output = model(src, trg)

        output = output[1:,].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        epoch_loss += loss.item()
    return epoch_loss/len(iterator)


def train(model, train_iter, val_iter, n_epochs, ignore_index, lr=0.002, clip=1):
    optimizer  = optim.Adam(model.parameters(),lr=lr)
    loss = nn.CrossEntropyLoss(ignore_index = ignore_index)
    for epoch in range(n_epochs):
        train_loss = train_step(model, train_iter, optimizer, loss, clip)
        val_loss = evaluate_step(model, val_iter, loss)
        print(f"Epoch: {epoch+1}-----------------------------------------------------")
        print(f"\tTrain Loss: {train_loss} | Train PPL:{math.exp(train_loss)}")
        print(f"\tVal.  Loss: {val_loss} | Val. PPL:{math.exp(val_loss)}")

In [260]:
train(model, train_iter,val_iter, 10, fr_vocab["<pad>"],0.001)

Epoch: 1-----------------------------------------------------
	Train Loss: 1.7827761709497199 | Train PPL:5.946341585950041
	Val.  Loss: 2.2731602558722863 | Val. PPL:9.710038583432159
Epoch: 2-----------------------------------------------------
	Train Loss: 1.712539669186552 | Train PPL:5.54302105650506
	Val.  Loss: 2.2463907996813455 | Val. PPL:9.453554418052404
Epoch: 3-----------------------------------------------------
	Train Loss: 1.6871250995154519 | Train PPL:5.403922612126288
	Val.  Loss: 2.27739780377119 | Val. PPL:9.75127264097394
Epoch: 4-----------------------------------------------------
	Train Loss: 1.6572673758642573 | Train PPL:5.244958743076166
	Val.  Loss: 2.2428354299985447 | Val. PPL:9.42000321608532
Epoch: 5-----------------------------------------------------
	Train Loss: 1.6363205080279255 | Train PPL:5.136235961684595
	Val.  Loss: 2.2248168740517054 | Val. PPL:9.251788406242223
Epoch: 6-----------------------------------------------------
	Train Loss: 1.6103

In [261]:
torch.save(model, "models/MNT_en-fr_v1.pkl")
#torch.save([en_vocab, fr_vocab], "models/MNT_en-fr_vocab.pkl")

## Prediction

In [187]:
from torchtext.data.metrics import bleu_score

In [262]:
def predict(model, src, src_tokenizer, src_vocab, trg_vocab, num_steps, device):
    model.eval()
    src_tokens = torch.tensor(src_vocab(src_tokenizer(src.lower())))
    src_input = torch.cat([torch.tensor([src_vocab["<bos>"]]), src_tokens, torch.tensor([src_vocab["<eos>"]])],dim=0)
    src_input = src_input.unsqueeze(1).to(device)
    encoder_outputs, hidden = model.encoder(src_input)
    dec_input = torch.tensor([trg_vocab["<bos>"]]).to(device)
    output_seq = []
    for _ in range(num_steps):
        Y, hidden = model.decoder(dec_input, hidden, encoder_outputs)
        dec_input = Y.argmax(dim=-1)
        pred = dec_input.squeeze().type(torch.int32).item()
        if pred==trg_vocab["<eos>"]:
            break
        output_seq.append(pred)
    return output_seq, trg_vocab.lookup_tokens(output_seq), " ".join(trg_vocab.lookup_tokens(output_seq))

In [266]:
candidates=[]
references=[]
for i, line in enumerate(sample(test_raw,5)):
    src, trg = line.split("\t")[:2]
    pred_ids, pred_tokens, pred_txt = predict(model, src, en_tokenizer,en_vocab, fr_vocab, 50, device)
    trg_tokens = fr_tokenizer(trg.lower())
    candidates.append(pred_tokens)
    references.append(trg_tokens)
    print(f"Test Case {i+1}:")
    print(f"Source text: {src}")
    print(f"Target text: {trg_tokens}")
    print(f"Translation: {pred_tokens}")
    ##print(f"BLEU Score: {bleu_score([pred_tokens], [trg_tokens])}")

Test Case 1:
Source text: I haven't left yet.
Target text: ['je', 'ne', 'suis', 'pas', 'encore', 'parti', '.']
Translation: ['je', "n'", 'ai', 'ai', 'encore', 'encore', 'encore', '.']
Test Case 2:
Source text: Tom is training.
Target text: ['tom', "s'", 'entraã', '®', 'ne', '.']
Translation: ['tom', "s'", 'entraã', '®', 't', '.']
Test Case 3:
Source text: Why did you try to run away?
Target text: ['pourquoi', 'avez', '-vous', 'essayã', '©', 'de', 'vous', 'enfuirâ€¯', '?']
Translation: ['pourquoi', 'as', '-', 'tu', 'essayã', '©', 'de', 'ã', '\xa0 ', 'enfuirâ€¯', '?']
Test Case 4:
Source text: Do you still believe I killed my brother?
Target text: ['crois', '-', 'tu', 'toujours', 'que', "j'", 'aie', 'tuã', '©', 'mon', 'frã¨re', '?']
Translation: ['croyez', '-vous', 'que', 'que', "j'", 'ai', 'tuã', '©', 'mon', 'frã¨re', '?']
Test Case 5:
Source text: Tom found that.
Target text: ['tom', 'a', 'trouvã', '©', 'cela', '.']
Translation: ['tom', 'a', 'trouvã', '©', 'couvert', '.']


# English to Mandarin Example

In [236]:
## Read all lines and shuffle
file = open("datasets/cmn-eng/cmn.txt", "r", encoding="UTF-8")
all_lines = file.readlines()
shuffle(all_lines)
#all_lines = sample(all_lines,100000)
print(len(all_lines))
## Train-Val-Test split
train_ratio, val_ratio = 0.8, 0.1
train_end = int(len(all_lines)*train_ratio)
val_end = int(len(all_lines)*(train_ratio+val_ratio))
train_raw = all_lines[:train_end]
val_raw = all_lines[train_end:val_end]
test_raw = all_lines[val_end:]

29371


In [238]:
## Use tokenizers from spacy
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
zh_tokenizer = get_tokenizer("spacy", language="zh_core_web_sm")
#zh_tokenizer = lambda x: list(x)
## Get source target vocabs
en_vocab, zh_vocab = get_vocabs(train_raw, en_tokenizer, zh_tokenizer)
## Get source tartget token ids
train_data = process_data(train_raw, en_tokenizer, zh_tokenizer, en_vocab, zh_vocab)
val_data = process_data(val_raw, en_tokenizer, zh_tokenizer, en_vocab, zh_vocab)
test_data = process_data(test_raw, en_tokenizer, zh_tokenizer, en_vocab, zh_vocab)
## Get train, val, test dataloader
collator = lambda x: generate_batch(x, en_vocab, zh_vocab)
bsz = 256
train_iter = DataLoader(train_data, batch_size=bsz, shuffle=True, collate_fn=collator)
val_iter = DataLoader(val_data, batch_size=bsz, shuffle=False, collate_fn=collator)
test_iter = DataLoader(test_data, batch_size=bsz, shuffle=False, collate_fn=collator)

In [243]:
input_dim = len(en_vocab)
emb_dim = 32
enc_hid_dim = 64
dec_hid_dim = 64
dropout = 0.8
encoder = Encoder(input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout)
attn_dim = 8
attention = Attention(enc_hid_dim, dec_hid_dim, attn_dim)
output_dim = len(zh_vocab)
decoder = Decoder(output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention)

##device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")  ## Only works on M-Series Mac
device = torch.device("cuda")## Only works on NVIDIA devices
model = Seq2Seq(encoder, decoder,device).to(device)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6518, 32)
    (rnn): GRU(32, 64, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=192, out_features=8, bias=True)
    )
    (embedding): Embedding(13607, 32)
    (rnn): GRU(160, 64)
    (out): Linear(in_features=224, out_features=13607, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
  )
)

In [244]:
train(model, train_iter,val_iter, 20, zh_vocab["<pad>"], lr=0.005)

Epoch: 1-----------------------------------------------------
	Train Loss: 6.1111171608385835 | Train PPL:450.8420970712251
	Val.  Loss: 5.490352710088094 | Val. PPL:242.3426684894301
Epoch: 2-----------------------------------------------------
	Train Loss: 5.2346368354299795 | Train PPL:187.66094216217022
	Val.  Loss: 5.226807316144307 | Val. PPL:186.1973841506843
Epoch: 3-----------------------------------------------------
	Train Loss: 4.94628021509751 | Train PPL:140.65079891700427
	Val.  Loss: 5.017607132593791 | Val. PPL:151.0494297512421
Epoch: 4-----------------------------------------------------
	Train Loss: 4.648452717324962 | Train PPL:104.42328816892052
	Val.  Loss: 4.861856301625569 | Val. PPL:129.2639323804961
Epoch: 5-----------------------------------------------------
	Train Loss: 4.359402013861614 | Train PPL:78.21035173056435
	Val.  Loss: 4.721812009811401 | Val. PPL:112.37168689891975
Epoch: 6-----------------------------------------------------
	Train Loss: 4.073

In [248]:
candidates=[]
references=[]
for i, line in enumerate(sample(test_raw,5)):
    src, trg = line.split("\t")[:2]
    pred_ids, pred_tokens, pred_txt = predict(model, src, en_tokenizer,en_vocab, zh_vocab, 50, device)
    trg_tokens = zh_tokenizer(trg.lower())
    candidates.append(pred_tokens)
    references.append(trg_tokens)
    print(f"Test Case {i+1}:")
    print(f"Source text: {src}")
    print(f"Target text: {trg_tokens}")
    print(f"Translation: {pred_tokens}")
    ##print(f"BLEU Score: {bleu_score([pred_tokens], [trg_tokens])}")

Test Case 1:
Source text: I think you're a really nice guy.
Target text: ['我', '認為', '你', '真的', '是', '一', '個', '好人', '。']
Translation: ['我', '觉得', '你', '真是', '个', '个', '好人', '。']
Test Case 2:
Source text: He is the same age as me.
Target text: ['他', '和', '我', '同岁', '。']
Translation: ['他', '對', '我', '同岁', '，', '我', '一样', '。', '。']
Test Case 3:
Source text: I think I shouldn't have asked Tom to do that.
Target text: ['我', '想', '我', '不', '该', '让', '汤姆', '做', '那', '件', '事', '。']
Translation: ['我', '認為', '我', '不', '該', '汤姆', '汤姆', '汤姆', '汤姆', '做', '。']
Test Case 4:
Source text: I wish I could go.
Target text: ['我', '希望', '我', '可以', '去', '。']
Translation: ['我', '願', '我', '能', '去', '。']
Test Case 5:
Source text: Tell him to mind his own business.
Target text: ['告訴', '他', '別多', '管', '閒事', '。']
Translation: ['他', '他', '向', '他', '的', '。', '。']
